In [7]:
!pip install langchain jq langchain-community langchain-openai

  Using cached tiktoken-0.8.0-cp312-cp312-macosx_10_13_x86_64.whl.metadata (6.6 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached jiter-0.8.2-cp312-cp312-macosx_10_12_x86_64.whl.metadata (5.2 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached regex-2024.11.6-cp312-cp312-macosx_10_13_x86_64.whl.metadata (40 kB)
Using cached tiktoken-0.8.0-cp312-cp312-macosx_10_13_x86_64.whl (1.0 MB)
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
Using cached jiter-0.8.2-cp312-cp312-macosx_10_12_x86_64.whl (302 kB)
Using cached regex-2024.11.6-cp312-cp312-macosx_10_13_x86_64.whl (288 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)


In [5]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

Enter API key for OpenAI:  ········


In [11]:
from langchain_community.document_loaders import JSONLoader
import json
from pathlib import Path

file_path='./opendatahub.json'
data = json.loads(Path(file_path).read_text())
loader = JSONLoader(
         file_path=file_path,
         jq_schema='.paths | to_entries[] | .key as $path | .value | to_entries[] | { path:$path, method:.key, tag:.value.tags[0], summary:.value.summary }',
         text_content=False)

In [12]:
docs = loader.load()

In [ ]:
len(docs)

In [ ]:
docs[0]

In [8]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [9]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [14]:
ids = vector_store.add_documents(documents=docs)

In [15]:
ids

['1431b31b-04cb-473e-9ab9-3302476d1dee',
 'fca0bc3b-0514-42bc-b627-6982f2e3a425',
 '08d7835d-b6d2-42d7-a2d1-392d8615dd8a',
 'c5fb6717-c8f1-4242-b286-514f6572e8b5',
 '6b3e8d99-f516-4d71-8440-6825b0af72c9',
 'eed822a8-fda9-498a-9ed3-4d8e3002033d',
 'd2b1a264-6f4d-454a-b5e5-92d5bfb7417c',
 '8a06e83f-59fb-4a4e-9229-6f8baec68bc2',
 '8f0a5edb-94b4-43bf-a83a-e07658b1e4e2',
 'c41ee878-db3a-4ea3-8f02-7adec6d32b0f',
 '77367f0d-7019-4ca8-9cc5-1383b7eb599e',
 'bf41ce12-028b-478c-b847-187b971cecda',
 'c7fb0727-68c3-420d-b5ee-f8fe86ffbc16',
 'ff65fada-9474-4e55-b551-c6c57b2a2337',
 '0fd811d0-c8b5-4af6-9825-807b1f345b51',
 'bfd917c6-b7ea-4e9e-91af-c5f1ed38a82a',
 'a2482b6a-29d2-4c19-8050-e1f4158ab023',
 '069b0907-841d-4685-b08b-46925fc262bc',
 'e2266b62-b1b5-4dc1-85a0-373c0bfc1f6c',
 'd31aa998-66e9-4704-8088-73b0b5d0d925',
 'f0a744ee-f6fa-4e83-afca-950732fde175',
 '08cd63bc-7cb8-4fb3-ab1f-0f0b89ad44f5',
 '6b7ab26d-4135-4177-9e59-6cf39d4d2402',
 'c6137d42-7cc2-4500-b43d-9f186be636ea',
 '2d64a65d-e92b-

In [31]:
question = "Given the information of the REST api provided above, write a piece of code to list the events in Python."

In [32]:
retrieved_docs = vector_store.similarity_search(question)
docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)
docs_content

'{"path": "/v2/Event", "method": "get", "tag": "EventV2", "summary": "GET Event List"}\n\n{"path": "/v1/Event", "method": "get", "tag": "Event", "summary": "GET Event List"}\n\n{"path": "/v1/EventTopics", "method": "get", "tag": "Event", "summary": "GET Event Topic List"}\n\n{"path": "/v1/EventShort", "method": "get", "tag": "EventShort", "summary": "GET EventShort List"}'

In [33]:
from langchain import hub
from langchain_openai import ChatOpenAI


llm = ChatOpenAI(model="gpt-4o-mini")

prompt = hub.pull("rlm/rag-prompt")

example_messages = prompt.invoke(
    {"context": "(context goes here)", "question": "(question goes here)"}
).to_messages()

prompt = prompt.invoke({"question": question, "context": docs_content})
answer = llm.invoke(prompt)
answer.pretty_print()

/Users/user/Projects/genai4se-course/env/lib/python3.12/site-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


================================== Ai Message ==================================

To list the events using the REST API in Python, you can use the `requests` library. Here's a sample code snippet:

```python
import requests

response = requests.get('https://yourapiurl.com/v2/Event')
events = response.json()
print(events)
```
